<a href="https://colab.research.google.com/github/SylviaNice/7162856-G-rez-Git-et-GitHub/blob/main/Projet_Bottleneck_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projet Python - Optimiser la gestion des données pour Bottleneck

In [79]:
# 1. Installation des bibliothèques

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
pd.set_option('display.max_columns', None)
from scipy.stats import zscore
import warnings
warnings.filterwarnings('ignore')

In [80]:
# 2. Importation des données

from google.colab import files
uploaded = files.upload()

Saving Fichier_erp.xlsx to Fichier_erp (1).xlsx
Saving fichier_liaison.xlsx to fichier_liaison (1).xlsx
Saving Fichier_web.xlsx to Fichier_web (1).xlsx


In [81]:
# 3. Chargement des 3 fichiers

erp_file = "Fichier_erp.xlsx"
web_file = "Fichier_web.xlsx"
liaison_file = "fichier_liaison.xlsx"

In [82]:
# 4. Lecture des fichiers

erp_df = pd.read_excel(erp_file)
web_df = pd.read_excel(web_file)
liaison_df = pd.read_excel(liaison_file)

In [143]:
# 5. Aperçu des premières lignes des fichiers importés

display(erp_df.head())
display(web_df.head())
display(liaison_df.head())

,product_id,onsale_web,price,stock_quantity,stock_status
0,3847,1,24.2,0,outofstock
1,3849,1,34.3,0,outofstock
2,3850,1,20.8,0,outofstock
3,4032,1,14.1,0,outofstock
4,4039,1,46.0,0,outofstock


,sku,virtual,downloadable,rating_count,average_rating,total_sales,tax_status,post_author,post_date,post_date_gmt,post_title,post_excerpt,post_status,comment_status,ping_status,post_name,post_modified,post_modified_gmt,post_parent,guid,menu_order,post_type,post_mime_type,comment_count
0,NaN,0,0,0,0.0,5.0,NaN,2.0,2018-06-07 16:27:25,2018-06-07 14:27:25,Château du Couvent Pomerol 2017,NaN,publish,closed,closed,chateau-du-couvent-pomerol-2017,2020-08-25 18:35:02,2020-08-25 16:35:02,0.0,https://www.bottle-neck.fr/wp-content/uploads/...,0.0,attachment,image/jpeg,0.0
1,NaN,0,0,0,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,0,0,0,0.0,3.0,taxable,2.0,2018-02-14 15:39:43,2018-02-14 14:39:43,Parés Baltà Penedès Indigena 2017,Des couleurs et aromes intenses où le fruit et...,publish,closed,closed,pares-balta-penedes-indigena-2017,2020-08-20 15:35:02,2020-08-20 13:35:02,0.0,https://www.bottle-neck.fr/?post_type=product&...,0.0,product,NaN,0.0
3,NaN,0,0,0,0.0,6.0,taxable,2.0,2018-02-14 17:15:31,2018-02-14 16:15:31,Maurel Cabardès Tradition 2017,"Un joli nez aux arômes de fruits rouges, de ca...",publish,closed,closed,maurel-cabardes-tradition-2017,2020-08-05 18:05:03,2020-08-05 16:05:03,0.0,https://www.bottle-neck.fr/?post_type=product&...,0.0,product,NaN,0.0
4,NaN,0,0,0,0.0,1.0,NaN,2.0,2020-01-25 14:08:16,2020-01-25 13:08:16,Domaine de la Monardière Vacqueyras Vieilles V...,NaN,publish,closed,closed,domaine-de-la-monardiere-vacqueyras-les-vieill...,2020-08-21 11:35:02,2020-08-21 09:35:02,0.0,https://www.bottle-neck.fr/wp-content/uploads/...,0.0,attachment,image/jpeg,0.0


,product_id,sku
0,3847,15298
1,3849,15296
2,3850,15300
3,4032,19814
4,4039,19815


In [84]:
# 6. Nombre de lignes et de colonnes de chacun des 3 fichiers

print(erp_df.shape)
print(web_df.shape)
print(liaison_df.shape)

(825, 5)
(1513, 28)
(825, 2)


In [85]:
# 7. Affichage du nom et du type de chaque colonne pour les fichiers erp, web et liaison

print(erp_df.dtypes)
print("-" * 40)

print(web_df.dtypes)
print("-" * 40)

print(liaison_df.dtypes)
print("-" * 40)

product_id          int64
onsale_web          int64
price             float64
stock_quantity      int64
stock_status       object
dtype: object
----------------------------------------
sku                              object
virtual                           int64
downloadable                      int64
rating_count                      int64
average_rating                  float64
total_sales                     float64
tax_status                       object
tax_class                       float64
post_author                     float64
post_date                datetime64[ns]
post_date_gmt            datetime64[ns]
post_content                    float64
post_title                       object
post_excerpt                     object
post_status                      object
comment_status                   object
ping_status                      object
post_password                   float64
post_name                        object
post_modified            datetime64[ns]
post_modified_g

In [145]:
# 8. Renommer la colonne (rectification de l'erreur de la stagiaire) et fusionner les 3 fichiers de données

liaison_df = pd.read_excel("fichier_liaison.xlsx")
liaison_df = liaison_df.rename(columns={'id_web': 'sku'})
print(liaison_df)

     product_id      sku
0          3847    15298
1          3849    15296
2          3850    15300
3          4032    19814
4          4039    19815
..          ...      ...
820        7203      NaN
821        7204      NaN
822        7247  13127-1
823        7329  14680-1
824        7338    16230

[825 rows x 2 columns]


In [146]:
# 9. Détecter les incohérences de prix : prix négatifs

erp_df[erp_df['price'] <= 0]

,product_id,onsale_web,price,stock_quantity,stock_status


In [136]:
# 10. Tranformation des valeurs négatives des prix en valeurs positives

erp_df['price'] = erp_df['price'].abs()
erp_df[erp_df['price'] <= 0]

,product_id,onsale_web,price,stock_quantity,stock_status


In [147]:
# 11. Affichage des données manquantes

print(liaison_df[liaison_df.isna().any(axis=1)])

     product_id  sku
19         4055  NaN
49         4090  NaN
50         4092  NaN
119        4195  NaN
131        4209  NaN
..          ...  ...
817        7196  NaN
818        7200  NaN
819        7201  NaN
820        7203  NaN
821        7204  NaN

[91 rows x 2 columns]


In [148]:
# 12. Détecter les valeurs manquantes dans les fichiers

print("Colonnes avec valeurs manquantes - ERP")
print(erp_df.isna().sum()[erp_df.isna().sum() > 0])
print("-" * 40)

print("Colonnes avec valeurs manquantes - Web")
print(web_df.isna().sum()
[web_df.isna().sum() > 0])
print("-" * 40)

print("Colonnes avec valeurs manquantes - Liaison")
print(liaison_df.isna().sum()
[liaison_df.isna().sum() > 0])
print("-" * 40)

Colonnes avec valeurs manquantes - ERP
Series([], dtype: int64)
----------------------------------------
Colonnes avec valeurs manquantes - Web
sku                  1431
average_rating          1
total_sales             1
tax_status            715
post_author             1
post_date               1
post_date_gmt           1
post_title              1
post_excerpt          715
post_status             1
comment_status          1
ping_status             1
post_name               1
post_modified           1
post_modified_gmt       1
post_parent             1
guid                    1
menu_order              1
post_type               1
post_mime_type        717
comment_count           1
dtype: int64
----------------------------------------
Colonnes avec valeurs manquantes - Liaison
sku    91
dtype: int64
----------------------------------------


In [151]:
# 13. Détecter le nombre de doublons pour chaque fichier

print(erp_df.duplicated().sum())
print(web_df.duplicated().sum())
print(liaison_df.duplicated().sum())


0
0
0


In [150]:
# 14. Visualisation des doublons

doublons_erp = erp_df.duplicated(keep=False)
doublons_web = web_df.duplicated(keep=False)
doublons_liaison = liaison_df.duplicated(keep=False)
print(doublons_erp, doublons_web, doublons_liaison)

0      False
1      False
2      False
3      False
4      False
       ...  
820    False
821    False
822    False
823    False
824    False
Length: 825, dtype: bool 0       False
1       False
2       False
3       False
4       False
        ...  
1426    False
1427    False
1428    False
1429    False
1430    False
Length: 1431, dtype: bool 0      False
1      False
2      False
3      False
4      False
       ...  
820    False
821    False
822    False
823    False
824    False
Length: 825, dtype: bool


In [152]:
# 15. Supprimer les doublons

erp_df.drop_duplicates(inplace=True)
web_df.drop_duplicates(inplace=True)
liaison_df.drop_duplicates(inplace=True)
print(len(erp_df), len(web_df), len(liaison_df))

825 1431 825


In [153]:
# 16. Vérification des doublons supprimés

print("Doublons ERP :", erp_df.duplicated().sum())
print("Doublons Web :", web_df.duplicated().sum())
print("Doublons Liaison :", liaison_df.duplicated().sum())

Doublons ERP : 0
Doublons Web : 0
Doublons Liaison : 0


In [154]:
# 17. Supprimer les lignes entièrement vides de mes fichiers

web_df.dropna(how='all', inplace=True)
erp_df.dropna(how='all', inplace=True)
liaison_df.dropna(how='all', inplace=True)
print(len(erp_df), len(web_df), len(liaison_df))

825 1431 825


In [155]:
# 18. Supprimer les 4 colonnes entièrement vides dans le fichier web

web_df.dropna(axis=1, how='all', inplace=True)
print(len(web_df.columns))

23


In [97]:
# 19. Vérification que les colonnes ont bien été supprimées

print(web_df.shape)

(1431, 24)


In [114]:
# Exemple : chargement du fichier
web_df = pd.read_excel("Fichier_web_clean.xlsx")

# Nom de la colonne sku, adaptatif
col_sku = 'sku' if 'sku' in web_df.columns else web_df.columns[0]

# Nettoyage : retirer espaces inutiles avant/après
web_df[col_sku] = web_df[col_sku].astype(str).str.strip()

# Garder uniquement les valeurs composées uniquement de chiffres
web_df[col_sku] = web_df[col_sku].where(web_df[col_sku].str.match(r'^\d+$'), np.nan)

# Sauvegarde
web_df.to_excel("Fichier_web_clean_checked.xlsx", index=False)
files.download("Fichier_web_clean_checked.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [107]:
# 20. Supression du texte dans la colonne sku (ne doivent se trouver que des chiffres)

# Définir le nom de la colonne SKU
col_sku = 'sku' if 'sku' in web_df.columns else web_df.columns[0]

# Définir le pattern : uniquement des chiffres
pattern = r'^\d+$'

# Remplacer les valeurs non conformes par NaN
web_df[col_sku] = web_df[col_sku].astype(str).where(
    web_df[col_sku].astype(str).str.match(pattern),
    other=np.nan
)
# Enregistrement du fichier nettoyé web
web_df.to_excel("Fichier_web_clean.xlsx", index=False)
files.download("Fichier_web_clean.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [99]:
# 21. Enregistrement des fichiers nettoyés erp et liaison

erp_df.to_excel("Fichier_erp_clean.xlsx", index=False)
liaison_df.to_excel("fichier_liaison_clean.xlsx", index=False)
files.download("Fichier_erp_clean.xlsx")
files.download("fichier_liaison_clean.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [100]:
# 22. Fusion des fichiers nettoyés

## Fusion du fichier erp avec le fichier liaison
erp_df = pd.read_excel("Fichier_erp_clean.xlsx")
liaison_df = pd.read_excel("fichier_liaison_clean.xlsx")
erp_liaison = pd.merge(erp_df, liaison_df, on='product_id', how='inner')
print(erp_liaison)
print(erp_liaison.shape)

     product_id  onsale_web  price  stock_quantity stock_status      sku
0          3847           1   24.2               0   outofstock    15298
1          3849           1   34.3               0   outofstock    15296
2          3850           1   20.8               0   outofstock    15300
3          4032           1   14.1               0   outofstock    19814
4          4039           1   46.0               0   outofstock    19815
..          ...         ...    ...             ...          ...      ...
820        7203           0   45.0              30      instock      NaN
821        7204           0   45.0               9      instock      NaN
822        7247           1   54.8              23      instock  13127-1
823        7329           0   26.5              14      instock  14680-1
824        7338           1   16.3              45      instock    16230

[825 rows x 6 columns]
(825, 6)


In [103]:
liaison_df.rename(columns={'SKU': 'sku'}, inplace=True)
print(liaison_df)

     product_id      sku
0          3847    15298
1          3849    15296
2          3850    15300
3          4032    19814
4          4039    19815
..          ...      ...
820        7203      NaN
821        7204      NaN
822        7247  13127-1
823        7329  14680-1
824        7338    16230

[825 rows x 2 columns]


In [140]:
print(liaison_df['sku'].value_counts().head(10))

print(web_df['sku'].value_counts().head(10))

sku
16230    1
15298    1
15296    1
15300    1
19814    1
19815    1
15303    1
14975    1
16042    1
14980    1
Name: count, dtype: int64
Series([], Name: count, dtype: int64)
